### Build a Simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "true")
HF_TOKEN = os.getenv("HF_TOKEN")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027496485D30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027496486900>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?"),
]

result = model.invoke(messages)

In [4]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to someone you know well)\n* **Coucou, ça va ?** (Very informal, to someone you\'re very close to)\n\n\nLet me know if you\'d like to know more about French greetings!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 21, 'total_tokens': 113, 'completion_time': 0.167272727, 'prompt_time': 0.003379155, 'queue_time': 0.25130091499999996, 'total_time': 0.170651882}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--984b1882-ca2a-46ba-b9c3-a47177f90a88-0', usage_metadata={'input_tokens': 21, 'output_tokens': 92, 'total_tokens': 113})

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to someone you know well)\n* **Coucou, ça va ?** (Very informal, to someone you\'re very close to)\n\n\nLet me know if you\'d like to know more about French greetings!\n'

In [6]:
chain = model | parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, used with friends and family)\n* **Coucou, ça va ?** (Very informal, used with close friends)\n\n\nLet me know if you\'d like to know more French greetings!\n'

In [7]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Trnaslate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [8]:
result = prompt.invoke({"language": "French", "text": "Hello"})

In [9]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [10]:
chain = prompt | model | parser
chain.invoke({"language": "French", "text": "Hello"})

'Bonjour \n'